MNIST_CNN_CLASS : accuracy about 99.3%

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)
tf.reset_default_graph() #변수 초기화
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


learning_rate = 0.001
training_epochs = 15
batch_size = 100


class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)

            self.x = tf.placeholder(tf.float32, [None, 784])
            
            x_img = tf.reshape(self.x, [-1, 28, 28, 1]) # image 28x28x1 (b/w)
            self.y = tf.placeholder(tf.float32, [None, 10])

            # Layer1 ImgIn shape=(?, 28, 28, 1)
            # Conv     -> (?, 28, 28, 32)
            # Pool     -> (?, 14, 14, 32)
            w1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
            l1 = tf.nn.conv2d(x_img, w1, strides=[1, 1, 1, 1], padding='SAME')
            l1 = tf.nn.relu(l1)
            l1 = tf.nn.max_pool(l1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            l1 = tf.nn.dropout(l1, keep_prob=self.keep_prob)
            '''
            Result:
            After Conv2D:0, shape=(?, 28, 28, 32)
            After Relu:0, shape=(?, 28, 28, 32)
            After MaxPool:0, shape=(?, 14, 14, 32)
            After dropout/mul:0, shape=(?, 14, 14, 32)
            '''

            # Layer 2 ImgIn shape=(?, 14, 14, 32)
            # Conv      ->(?, 14, 14, 64)
            # Pool      ->(?, 7, 7, 64)
            w2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
            l2 = tf.nn.conv2d(l1, w2, strides=[1, 1, 1, 1], padding='SAME')
            l2 = tf.nn.relu(l2)
            l2 = tf.nn.max_pool(l2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            l2 = tf.nn.dropout(l2, keep_prob=self.keep_prob)
            '''
            Result:
            After Conv2D_1:0, shape=(?, 14, 14, 64)
            After Relu_1:0, shape=(?, 14, 14, 64)
            After MaxPool_1:0, shape=(?, 7, 7, 64)
            After dropout_1/mul:0, shape=(?, 7, 7, 64)
            '''

            # Layer 3 ImgIn shape=(?, 7, 7, 64)
            # Conv      ->(?, 7, 7, 128)
            # Pool      ->(?, 4, 4, 128)
            # Reshape   ->(?, 4 * 4 * 128)
            w3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
            l3 = tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME')
            l3 = tf.nn.relu(l3)
            l3 = tf.nn.max_pool(l3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            l3 = tf.nn.dropout(l3, keep_prob=self.keep_prob)
            l3_flat = tf.reshape(l3, [-1, 128 * 4 * 4])
            '''
            Result:
            After Conv2D_2:0, shape=(?, 7, 7, 128)
            After Relu_2:0, shape=(?, 7, 7, 128)
            After MaxPool_2:0, shape=(?, 4, 4, 128)
            After dropout_2/mul:0, shape=(?, 4, 4, 128)
            After Reshape_1:0, shape=(?, 2048)
            '''

            # Layer 4 FC 4x4x128 inputs -> 625 outputs
            w4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625], initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            l4 = tf.nn.relu(tf.matmul(l3_flat, w4) + b4)
            l4 = tf.nn.dropout(l4, keep_prob=self.keep_prob)
            '''
            Result:
            After Relu_3:0, shape=(?, 625)
            After dropout_3/mul:0, shape=(?, 625)
            '''

            # L5 Final FC 625 inputs -> 10 outputs
            w5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.logits = tf.matmul(l4, w5) + b5
            '''
            Result:
            After add_1:0, shape=(?, 10), dtype=float32)
            '''

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, keep_prop=1.0):
        return self.sess.run(self.logits, feed_dict={self.x: x_test, self.keep_prob: keep_prop})

    def get_accuracy(self, x_test, y_test, keep_prop=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.x: x_test, self.y: y_test, self.keep_prob: keep_prop})

    def train(self, x_data, y_data, keep_prop=0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.x: x_data, self.y: y_data, self.keep_prob: keep_prop})

sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning Started')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Finished')

print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning Started
Epoch: 0001 cost = 0.401218228
Epoch: 0002 cost = 0.093768328
Epoch: 0003 cost = 0.066042913
Epoch: 0004 cost = 0.060063882
Epoch: 0005 cost = 0.050105646
Epoch: 0006 cost = 0.044522589
Epoch: 0007 cost = 0.042557313
Epoch: 0008 cost = 0.037961045
Epoch: 0009 cost = 0.036576295
Epoch: 0010 cost = 0.034148116
Epoch: 0011 cost = 0.031219186
Epoch: 0012 cost = 0.028509897
Epoch: 0013 cost = 0.029053167
Epoch: 0014 cost = 0.026715464
Epoch: 0015 cost = 0.026679610
Finished
Accuracy: 0.9928


MNIST_CNN_LAYERS : accuracy about 99.3%

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)
tf.reset_default_graph() #변수 초기화
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


learning_rate = 0.001
training_epochs = 15
batch_size = 100


class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)

            self.x = tf.placeholder(tf.float32, [None, 784])

            x_img = tf.reshape(self.x, [-1, 28, 28, 1])
            self.y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,rate=0.3, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], padding="same", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,rate=0.3, training=self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat,units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,rate=0.5, training=self.training)

            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,feed_dict={self.x: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.x: x_test,self.y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.x: x_data, self.y: y_data, self.training: training})

sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning Started')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Finished')

print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Learning Started
Epoch: 0001 cost = 0.286512199
Epoch: 0002 cost = 0.087843931
Epoch: 0003 cost = 0.066050227
Epoch: 0004 cost = 0.057555984
Epoch: 0005 cost = 0.051219282
Epoch: 0006 cost = 0.045764082
Epoch: 0007 cost = 0.041590734
Epoch: 0008 cost = 0.037859480
Epoch: 0009 cost = 0.037435320
Epoch: 0010 cost = 0.033222694
Epoch: 0011 cost = 0.032709280
Epoch: 0012 cost = 0.031441033
Epoch: 0013 cost = 0.028789883
Epoch: 0014 cost = 0.028018515
Epoch: 0015 cost = 0.027397597
Fi

MNIST_CNN_ENSEMBLES : accuracy about 99.5%

In [0]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777) 
tf.reset_default_graph() #변수 초기화
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 20
batch_size = 100


class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)

            self.x = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white), Input Layer
            x_img = tf.reshape(self.x, [-1, 28, 28, 1])
            self.y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,rate=0.3, training=self.training)

            # Convolutional Layer #2 and Pooling Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,rate=0.3, training=self.training)

            # Convolutional Layer #3 and Pooling Layer #3
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],padding="SAME", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,rate=0.3, training=self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat,units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,rate=0.5, training=self.training)

            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,feed_dict={self.x: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.x: x_test,self.y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.x: x_data, self.y: y_data, self.training: training})

sess = tf.Session()

models = []
num_models = 2
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started')

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Finished')

test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning Started
Epoch: 0001 cost = [0.28585465 0.28938861]
Epoch: 0002 cost = [0.09087616 0.08951245]
Epoch: 0003 cost = [0.06626785 0.06888541]
Epoch: 0004 cost = [0.05536487 0.05645393]
Epoch: 0005 cost = [0.05237273 0.05150979]
Epoch: 0006 cost = [0.04593779 0.04589141]
Epoch: 0007 cost = [0.03936706 0.04024985]
Epoch: 0008 cost = [0.03643613 0.03556717]
Epoch: 0009 cost = [0.03475948 0.03385207]
Epoch: 0010 cost = [0.03444355 0.03349655]
Epoch: 0011 cost = [0.03271658 0.03139096]
Epoch: 0012 cost = [0.03056721 0.02934574]
Epoch: 0013 cost = [0.02890254 0.02863802]
Epoch: 0014 cost = [0.0269985  0.02800886]
Epoch: 0015 cost = [0.02676732 0.02515708]
Epoch: 0016 cost = [0.02539744 0.02717334]
Epoch: 0017 cost = [0.02388613 0.02651006]
Epoch: 0018 cost = [0.0251159  0.02462279]
Epoch: 0019 cos